# Earth System Simulation - Google Colab Version

This notebook is optimized for running the Earth system simulation in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/earth_system_sim/blob/main/examples/colab_simulation.ipynb)

## 1. Setup Environment

First, let's set up our environment and install dependencies.

In [ ]:
# Clone repository
!git clone https://github.com/yourusername/earth_system_sim.git
%cd earth_system_sim

# Install requirements
!pip install -r requirements.txt

# Add project root to Python path
import sys
from pathlib import Path
sys.path.append(str(Path.cwd()))

## 2. Mount Google Drive (Optional)

If you want to save results persistently, mount your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create output directory in Drive
import os
output_dir = '/content/drive/MyDrive/earth_system_results'
os.makedirs(output_dir, exist_ok=True)

## 3. Import Dependencies

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml
from IPython.display import HTML
from tqdm.notebook import tqdm

from scripts.run_simulation import EarthSystemSimulation
from scripts.visualize_results import create_visualizations

## 4. Check GPU Availability

Verify that we have access to a GPU.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

## 5. Load and Modify Configuration

Load the default configuration and adjust for Colab's resources.

In [ ]:
# Load config
with open('config/model_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Adjust for Colab resources
config['grid_height'] = 16  # Reduced size
config['grid_width'] = 16   # Reduced size

# Save modified config
with open('config/colab_config.yaml', 'w') as f:
    yaml.dump(config, f)

## 6. Run Simulation

In [ ]:
# Create simulation
sim = EarthSystemSimulation('config/colab_config.yaml', device)

# Run simulation
print("Running simulation...")
trajectory = sim.run_simulation(
    num_steps=50,
    save_frequency=5
)

print("Simulation complete!")

## 7. Create Visualizations

In [ ]:
# Load visualization config
with open('config/visualization_config.yaml', 'r') as f:
    viz_config = yaml.safe_load(f)

# Create visualizations
print("Creating visualizations...")
create_visualizations(
    trajectory,
    viz_config,
    output_dir,
    make_animations=True
)

print(f"Results saved to: {output_dir}")

## 8. Display Interactive Results

Create and display interactive visualizations of the results.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create interactive dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Temperature Field',
        'Vegetation Distribution',
        'Topography',
        'System Evolution'
    )
)

# Add temperature plot
fig.add_trace(
    go.Heatmap(
        z=trajectory['physical'][0][..., 0],
        colorscale='RdBu_r',
        name='Temperature'
    ),
    row=1, col=1
)

# Add vegetation plot
fig.add_trace(
    go.Heatmap(
        z=trajectory['biosphere'][0][..., 0],
        colorscale='YlGn',
        name='Vegetation'
    ),
    row=1, col=2
)

# Add topography plot
fig.add_trace(
    go.Surface(
        z=trajectory['geosphere'][0][..., 0],
        colorscale='terrain',
        name='Elevation'
    ),
    row=2, col=1
)

# Add time series
times = np.array([t['physical'] for t in trajectory['times']])
temp_mean = np.mean(trajectory['physical'][..., 0], axis=(1, 2))

fig.add_trace(
    go.Scatter(
        x=times,
        y=temp_mean,
        name='Mean Temperature',
        line=dict(color='red')
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    width=1000,
    title_text="Earth System Simulation Results",
    showlegend=True
)

fig.show()

## 9. Save Results to Drive

In [ ]:
# Save trajectory data
data_path = os.path.join(output_dir, 'simulation_results.npz')
np.savez(
    data_path,
    physical_states=trajectory['physical'],
    biosphere_states=trajectory['biosphere'],
    geosphere_states=trajectory['geosphere'],
    times=trajectory['times']
)

print(f"Results saved to: {data_path}")